In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import math
import datetime

# in a notebook, load the tensorboard extension, not needed for scripts
%load_ext tensorboard

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 3GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=3072)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

c:\Users\Nikod\miniconda3\envs\iannwtf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
(train_ds, test_ds), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [5]:
def preprocess(data, batch_size):
    # image should be float
    data = data.map(lambda x, t: (tf.cast(x, tf.float32), t))
    # image vector will here have values between -1 and 1
    data = data.map(lambda x,t: ((x/255, t)))
    # batch the dataset
    data = data.batch(batch_size)
    # prefetch
    data = data.prefetch(tf.data.AUTOTUNE)
    return data

train_ds = preprocess(train_ds, 64)
test_ds = preprocess(test_ds,64)

In [6]:
# Build the model using the functional API
# input layer
i = tf.keras.Input(shape=[32,32,3])
# i = tf.keras.Input(shape=(3074))
# your code here, just add convolutional layers

conv_1 = tf.keras.layers.Conv1D(64, 3, activation="relu")(i)
conv_2 = tf.keras.layers.Conv1D(64, 3, activation="relu")(conv_1)
conv_3 = tf.keras.layers.Conv1D(64, 3, activation="relu")(conv_2)


# pooling layer to get 1D data
x = tf.keras.layers.GlobalAveragePooling2D()(conv_3)

# last hidden layer i.e.. output layer
y = tf.keras.layers.Dense(10, activation='softmax')(x)
 
model = tf.keras.Model(i, y)
 
# model description
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv1d (Conv1D)             (None, 32, 30, 64)        640       
                                                                 
 conv1d_1 (Conv1D)           (None, 32, 28, 64)        12352     
                                                                 
 conv1d_2 (Conv1D)           (None, 32, 26, 64)        12352     
                                                                 
 global_average_pooling2d (G  (None, 64)               0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                650       
                                                             

In [7]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

In [8]:
# Fit
# You could also experiment around with different hyper parameter, but this configuration should work fine
num_epochs = 8
history = model.fit(train_ds, validation_data = test_ds,  epochs=num_epochs)

Epoch 1/8
782/782 [==============================] - 156s 196ms/step - loss: 2.0348 - accuracy: 0.2317 - val_loss: 1.9351 - val_accuracy: 0.2763
Epoch 2/8
782/782 [==============================] - 165s 211ms/step - loss: 1.9074 - accuracy: 0.2886 - val_loss: 1.8477 - val_accuracy: 0.3213
Epoch 3/8
782/782 [==============================] - 159s 204ms/step - loss: 1.8421 - accuracy: 0.3235 - val_loss: 1.7980 - val_accuracy: 0.3418
Epoch 4/8
782/782 [==============================] - 159s 204ms/step - loss: 1.7946 - accuracy: 0.3476 - val_loss: 1.7572 - val_accuracy: 0.3639
Epoch 5/8
782/782 [==============================] - 154s 197ms/step - loss: 1.7589 - accuracy: 0.3660 - val_loss: 1.7242 - val_accuracy: 0.3818
Epoch 6/8
782/782 [==============================] - 149s 191ms/step - loss: 1.7311 - accuracy: 0.3800 - val_loss: 1.6993 - val_accuracy: 0.3934
Epoch 7/8
782/782 [==============================] - 149s 190ms/step - loss: 1.7086 - accuracy: 0.3908 - val_loss: 1.6814 - val_ac